In [18]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1f7uslI-ZHidriQFZR966_aILjlkgDN76',
dest_path='content/eye_gender_data.zip',
unzip=True)

Unzipping...Done.


In [19]:
!pip install visualkeras

     |████████████████████████████████| 795 kB 7.4 MB/s 


In [52]:
import pandas as pd # Data analysis and manipultion tool
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf # Deep Learning Tool
import os # OS module in Python provides a way of using operating system dependent functionality
import cv2 # Library for image processing
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set
from sklearn.metrics import f1_score
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from google.colab.patches import cv2_imshow
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout,GlobalAveragePooling2D

In [60]:
labels = pd.read_csv("/content/content/eye_gender_data/Training_set.csv") # loading the labels
file_paths = [[fname, '/content/content/eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

x_train = []
y_train = [] # initialize an empty numpy array
image_size = 128 # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):

  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
  x_train.append(new_img_array)
  y_train.append(train_data['label'][i])

In [61]:

x_train=np.reshape(x_train,(len(train_data),128,128,1))
x_train.shape

(9220, 128, 128, 1)

In [63]:
datagen=ImageDataGenerator(rescale=1./255,horizontal_flip=True,validation_split=0.3)

In [64]:
lab=LabelEncoder()
lab.fit(y_train)
lab.classes_
y_train = lab.transform(y_train)
y_train


array([1, 0, 0, ..., 1, 1, 0])

In [65]:
train_generator=datagen.flow(x_train,y_train,batch_size=32,shuffle=True,subset='training')

In [66]:
validation_generator=datagen.flow(x_train,y_train,batch_size=32,shuffle=True,subset='validation')

In [67]:
tf.keras.backend.clear_session()

In [68]:
INPUT_SHAPE = (128, 128, 1)

# define sequential model
model = Sequential()
# define conv-pool layers - set 1
model.add(Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1),
activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))
# define conv-pool layers - set 2
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1),
activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add flatten layer
model.add(tf.keras.layers.Flatten())

# add dense layers with some dropout
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))

# add output layer
model.add(Dense(1, activation='sigmoid'))

In [69]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               7373056   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0

In [70]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [71]:
step_size_train=train_generator.n//train_generator.batch_size
print(step_size_train)
step_size_validation=validation_generator.n//validation_generator.batch_size
print(step_size_validation)

model.fit(train_generator,validation_data=validation_generator,epochs=7,steps_per_epoch=step_size_train,validation_steps=step_size_validation)

201
86
Epoch 1/7
201/201 [==============================] - 81s 397ms/step - loss: 0.5771 - accuracy: 0.6865 - val_loss: 0.4320 - val_accuracy: 0.8107
Epoch 2/7
201/201 [==============================] - 79s 396ms/step - loss: 0.4431 - accuracy: 0.7987 - val_loss: 0.4343 - val_accuracy: 0.8056
Epoch 3/7
201/201 [==============================] - 80s 396ms/step - loss: 0.3822 - accuracy: 0.8334 - val_loss: 0.3730 - val_accuracy: 0.8379
Epoch 4/7
201/201 [==============================] - 80s 398ms/step - loss: 0.3526 - accuracy: 0.8499 - val_loss: 0.3908 - val_accuracy: 0.8354
Epoch 5/7
201/201 [==============================] - 80s 397ms/step - loss: 0.3277 - accuracy: 0.8594 - val_loss: 0.3469 - val_accuracy: 0.8467
Epoch 6/7
201/201 [==============================] - 80s 398ms/step - loss: 0.3023 - accuracy: 0.8739 - val_loss: 0.3037 - val_accuracy: 0.8761
Epoch 7/7
201/201 [==============================] - 80s 398ms/step - loss: 0.2864 - accuracy: 0.8818 - val_loss: 0.3159 - val_ac

In [72]:
model.evaluate(train_generator)

202/202 [==============================] - 21s 103ms/step - loss: 0.2384 - accuracy: 0.9025


[0.23842072486877441, 0.9025410413742065]

In [73]:
model.evaluate(validation_generator)

87/87 [==============================] - 9s 102ms/step - loss: 0.3169 - accuracy: 0.8662


[0.3169477880001068, 0.8662328124046326]

In [75]:
labels = pd.read_csv("/content/content/eye_gender_data/Testing_set.csv") # loading the labels
file_paths = [[fname, '/content/content/eye_gender_data/test/' + fname] for fname in labels['filename']]
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_data = pd.merge(images, labels, how = 'inner', on = 'filename')

x_test = []
image_size = 128 # image size taken is 100 here. one can take other size too
for i in range(len(test_data)):

  img_array = cv2.imread(test_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
  x_test.append([new_img_array])

In [76]:
x_test=np.reshape(x_test,(len(test_data),128,128,1))

In [77]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [78]:
test_generator=test_datagen.flow(x_test,batch_size=32,shuffle=False)

In [79]:
predictions=model.predict(test_generator)
predictions=np.reshape(predictions,-1)
predictions.shape

(2305,)

In [80]:
predictions = (predictions>0.4).astype(int)
predictions=pd.DataFrame(predictions,columns=['label'])
predictions

,label
0,1
1,0
2,0
3,0
4,1
...,...
2300,0
2301,0
2302,0
2303,1


In [81]:
predictions['label'].replace([0,1],['female','male'],inplace=True)
predictions

,label
0,male
1,female
2,female
3,female
4,male
...,...
2300,female
2301,female
2302,female
2303,male


In [82]:
predictions=predictions.to_numpy()
predictions=np.reshape(predictions,-1)
predictions

array(['male', 'female', 'female', ..., 'female', 'male', 'female'],
      dtype=object)

In [83]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)

# To download the csv file locally
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>